<a href="https://colab.research.google.com/github/Alecia113/NLP-Lab/blob/main/lab4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recurrent Neural Networks + Sequence Modelling

In [3]:
#recurrent （复发的 蕊·科·run3·忒） sequence （计数，序数） （C 困·嘶） neural（牛肉）
#RNN ； seq2seq
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# you can GPU(cuda)/ just CPU

#RNN

In [2]:
'''
RNN 是一类神经网络。其中节点之间的连接形成沿着时间序列的有向图。 
时间手段跨时间分布。
将它与分布在空间中的网络进行比较。这可让他表现出时间动态行为。
RNN 源自前馈神经网络。
可以使用其内部状态（内存）来处理可变长度的输入序列

'''

'\nRNN 是一类神经网络。其中节点之间的连接形成沿着时间序列的有向图。 \n时间手段跨时间分布。\n将它与分布在空间中的网络进行比较。这可让他表现出时间动态行为。\nRNN 源自前馈神经网络。\n可以使用其内部状态（内存）来处理可变长度的输入序列\n\n'

#**predict the last character of the word** 预测单词的最后一个字符

**eg："wor"-->"d", "dee"-->"p"**

##有什么用呢？？？？？？

In [3]:
#这个模块的用意呢：输入单词。输入除了最后一个看，看能不能准确的匹配上最后一位

'''
1导包
2导入文本库 tip：有，直接换行就好了  [字母表]
encoding 编码 decoding解码  num_dic 让word 和index 对应起来
看下长度26
4这才开始导入文本。你想做测试的文本

5写函数了，开始要做拆解了(批量处理)
目的

# Make a batch to have sequence data for input and ouput
# wor -> X, d -> Y
# dee -> X, p -> Y

6开始循环，把除了最后一个【：-1】的字母的index都取出来
            # input data is:
        # wor       woo       dee     div      ...
        # [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...


    再把最后一个单词取出来，把index 放到target里

    #制作one-hot np.eye 批量制作。然后第一个是0.所以index+1的位置是1.代表了这个元素
【将输入转为了一键编码】就不像之前我还得一个个编进去用*vec【0】
 # convert input to one-hot encoding.
        # if input is [3, 4, 4]:
        # [[ 0,  0,  0,  1,  0,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]
        #  [ 0,  0,  0,  0,  1,  0,  0, ... 0]]
'''

'''
数据准备完开始设置hyerparameters
'''

'''
然后准备dropout 是一个全连接层，开始丢弃神经元了
'''

'\n然后准备dropout 是一个全连接层，开始丢弃神经元了\n'

In [4]:
#这个过程的数据准备还是很像CBOW的

import numpy as np
char_arr = ['a','b','c','d','e',
            'f','g','h','i','j',
            'k','l','m','n','o',
            'p','q','r','s','t',
            'u','v','w','x','y','z']

num_dic = {n:i for i,n in enumerate(char_arr)}  # {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
dic_len = len(num_dic)

seq_data = ['word','wood','deep','dive','cold',
            'cool','load','love','kiss','kind']

def make_batch(seq_data):
  input_batch = []
  target_batch = []

  for seq in seq_data:
    input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来
    target = num_dic[seq[-1]]

    input_batch.append(np.eye(dic_len)[input_data])

    target_batch.append([target])  #就是输入的index ；target要加【】

  return input_batch, target_batch

#这个过程的数据准备还是很像CBOW的


In [5]:
# setting hyperparameters
learning_rate = 0.01
n_hidden = 64
total_epoch = 50

n_step = 3    #暂时的RNN序列长度 temporal length of sequences for RNN

n_input = dic_len  #26 输入向量的维度（dimension of input vector)
n_class = dic_len #类的数量 26 number of class



**dropout 让每个隐藏单元更强大； 并推动其自行创建有用的功能。不用依赖其他隐藏单元来纠正错误**

In [6]:
#感觉像是从一个全连接开始丢神经元。（还是全连接）
'''
1调包
建类别 调用nn模型开始建设RNN
    讲解RNN中参数： 
          batch_first 默认（default）False ； 代表输入和输出张量的提供方式为（seq_len, batch_size, feature)
          但是：我们需要True ：bec：我们要使用形状输入（batch_size, seq_len, feature)
            设置 num_layer = 2 我们将两个RNN 层堆叠在一起形成一个堆叠的RNN。
                              第一个RNN接收第一个RNN的输出，并计算最终结果。
          应用dropout（=0.2） aim：防止过拟合； 可更改 ； 【注意】可以用在RNN的每一层除了最后一层
          https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
          官方解读

  forward function（函数）
   nn.RNN 有两个output
  1：形状张量（batch_size, seq_len, hidden_size)有点像True；包含每个时间步长t 的RNN的最后一层输出特征。
  2： 形状张量（num_layers * num_directions, batch, hidden_size) :包含 t = seq_len 的隐藏状态的张量
   我们只关心第一个输出
    nn.RNN: https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN


    我们仅仅要RNN输出功能中提取最后一个隐藏状态（from output features get(extract) last hidden state)
    最后的隐藏状态将信息从RNN信元（cell）随时间流逝中移走
    因此    基于最后一个隐藏状态的预测，不仅考虑当前时间'  步长的数据，还要考虑历史数据。（就是输出中包含很多类别的数据）
    来源复杂

输入然后得到的输出项（forward）

  3:设置模型，到GPU
  4: loss + optimizer

二

5模型建设完，开始准备输入数据
#https://blog.csdn.net/u014687517/article/details/90770132?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161698050316780265445973%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161698050316780265445973&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-2-90770132.first_rank_v2_pc_rank_v29&utm_term=.view的用途
view 用法。
view 很像是reshape
a = torch.range(1, 25) #a是长度25的张量
#改成了5*5：a = a.view(5, 5)
或者：a = a.view(-1, 5)

（感觉就是将数据类型转变成为了张量。再传参一波，没做什么其他的感觉）

三
开始训练了epoch
然后经典三部 forward + backward + optimize
  #神经网络中经常会有
  作用:output 是通过神经网络最后一层softmax 函数作用之后的输出。
  
  例如：假设我们在分类问题中共有4类。 分别是0123； 假如output.data 第一行【1，000】（属于第一个类别的概率是1
  不加_, :输出则是1， （其中1是tensor） 加了之后输出变成0

  下划线+'，'的作用是：使预测返回的output.data行中最大数值所在位置代表的类别。
  _,predicted=torch.max(output.data,dim=1)

  【另】
    torch.max()返回两个值，一个是具体的value ；用下划线表示。第二个值是 value对应的index（predicted）
    下划线_ 表示的就是具体的value，也就是输出的最大值（可用其他名词代替下划线）
    数字1其实可以写为dim=1，这里简写为1
    dim = 1 输出行的最大值。 dim = 0 输出列的最大值
    #https://blog.csdn.net/weixin_48249563/article/details/111387501?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-1.control&dist_request_id=1328740.37292.16169815268198889&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-1.control


结论是损失和 训练准确度

四：
预测
设置flag 去计算模型； 会 turn off ：dropout 
将标志设置为评估模式，这将dropout退出。

训练完train_datasets之后，model要来测试样本了。在model(test_datasets)之前，需要加上model.eval(). 否则的话，有输入数据，即使不训练，它也会改变权值。这是model中含有batch normalization层所带来的的性质。
在做one classification的时候，训练集和测试集的样本分布是不一样的，尤其需要注意这一点。

[训练完数据要训练样本；为了让训练集和测试集的样本分开]（感觉更像是实现全连接的需要的感觉）
#https://blog.csdn.net/qq_38410428/article/details/101102075?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161698276116780271557000%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161698276116780271557000&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_click~default-2-101102075.first_rank_v2_pc_rank_v29&utm_term=model.eval

先把预测值取出来。
然后循环起来

pytorch中GPU与CPU的相互转化

深度学习中我们默认使用的是CPU，如果我们要使用GPU，需要使用.cuda将计算或者数据从CPU移动至GPU，

如果当我们需要在CPU上进行运算时，比如使用plt可视化绘图, 我们可以使用.cpu将计算或者数据转移至CPU.

然后开始拼接单词。
看拼出来的单词对不对

'''

"\n1调包\n建类别 调用nn模型开始建设RNN\n    讲解RNN中参数： \n          batch_first 默认（default）False ； 代表输入和输出张量的提供方式为（seq_len, batch_size, feature)\n          但是：我们需要True ：bec：我们要使用形状输入（batch_size, seq_len, feature)\n            设置 num_layer = 2 我们将两个RNN 层堆叠在一起形成一个堆叠的RNN。\n                              第一个RNN接收第一个RNN的输出，并计算最终结果。\n          应用dropout（=0.2） aim：防止过拟合； 可更改 ； 【注意】可以用在RNN的每一层除了最后一层\n          https://pytorch.org/docs/stable/generated/torch.nn.RNN.html\n          官方解读\n\n  forward function（函数）\n   nn.RNN 有两个output\n  1：形状张量（batch_size, seq_len, hidden_size)有点像True；包含每个时间步长t 的RNN的最后一层输出特征。\n  2： 形状张量（num_layers * num_directions, batch, hidden_size) :包含 t = seq_len 的隐藏状态的张量\n   我们只关心第一个输出\n    nn.RNN: https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN\n\n\n    我们仅仅要RNN输出功能中提取最后一个隐藏状态（from output features get(extract) last hidden state)\n    最后的隐藏状态将信息从RNN信元（cell）随时间流逝中移走\n    因此    基于最后一个隐藏状态的预测，不仅考虑当前时间'  步长的数据，还要考虑历史数据。（就是输出中包含很多类别的数据）\n    来源复杂\n\n输入然后得到的输出项（forward）\n\

#Model

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
#感觉固定住了

class RNN_Model(nn.Module):
  def __init__(self):
    super(RNN_Model, self).__init__()
    self.rnn = nn.RNN(n_input, n_hidden, num_layers=2, batch_first=True, dropout=0.2)
    self.linear = nn.Linear(n_hidden, n_class) #output的线性层

  def forward(self, x):
    rnn_output, h_n = self.rnn(x)
    x_last = rnn_output[:,-1,:]
    x = self.linear(x_last)
    return x

model = RNN_Model().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr= learning_rate)
#模型建立好了
#然后准备输入
input_batch, target_batch = make_batch(seq_data)
input_batch_torch = torch.from_numpy(np.array(input_batch)).float().to(device)  #把一个单词前三的字母的one-hot 张量形式放到一个list里面
target_batch_torch = torch.from_numpy(np.array(target_batch)).view(-1).to(device) #把每个单元都整合成了长单元
# to(device) 将输入转换为张量，并将其设置成为GPU 
# 参数-1 含义：自行计算行数或列数 


for epoch in range(total_epoch):
  model.train()  #将标志设置为训练模式；

  optimizer.zero_grad()
  outputs = model(input_batch_torch)    #output训练
  loss = criterion(outputs, target_batch_torch)
  loss.backward()
  optimizer.step()
  _, predicted = torch.max(outputs,1) #这个_代表的是什么

  acc = accuracy_score(predicted.cpu().numpy(),target_batch_torch.cpu().numpy()) #50个数据 epoch
  print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch+1, loss.item(),acc))

print('Finished Training')

##Prediction
model.eval()
outputs = model(input_batch_torch)
_, predicted = torch.max(outputs, 1)    # same as predicted = torch.argmax(pred_outputs, 1)

predict_words = []
for i in range(len(predicted.cpu().numpy())):
  ind = predicted.cpu().numpy()[i] #这步骤是让他变回数组而不是张量然后分别取出预测的每个值' index
  predict_words.append(seq_data[i][:-1]+char_arr[ind])  #他是去你除了最后一位的单词组合，然后将你预测的index放到对应的字母表中。从字母表中找到真实值。再将两个拼接
  print(predict_words)

print('\n=== Prediction Result ===')
print('Input:', [w[:3] + ' ' for w in seq_data])
print('Predicted:', predict_words)
print('Accuracy: %.2f' %acc)

Epoch: 1, loss: 3.19482, train_acc: 0.00
Epoch: 2, loss: 2.61596, train_acc: 0.50
Epoch: 3, loss: 1.93472, train_acc: 0.50
Epoch: 4, loss: 1.57602, train_acc: 0.50
Epoch: 5, loss: 1.41090, train_acc: 0.50
Epoch: 6, loss: 1.28457, train_acc: 0.50
Epoch: 7, loss: 1.34090, train_acc: 0.60
Epoch: 8, loss: 1.30429, train_acc: 0.60
Epoch: 9, loss: 1.24063, train_acc: 0.50
Epoch: 10, loss: 1.05597, train_acc: 0.70
Epoch: 11, loss: 0.96480, train_acc: 0.70
Epoch: 12, loss: 0.83078, train_acc: 0.70
Epoch: 13, loss: 0.85647, train_acc: 0.60
Epoch: 14, loss: 0.67300, train_acc: 0.70
Epoch: 15, loss: 0.59613, train_acc: 0.80
Epoch: 16, loss: 0.54070, train_acc: 0.80
Epoch: 17, loss: 0.42144, train_acc: 0.90
Epoch: 18, loss: 0.36935, train_acc: 0.90
Epoch: 19, loss: 0.33272, train_acc: 0.90
Epoch: 20, loss: 0.28066, train_acc: 1.00
Epoch: 21, loss: 0.24454, train_acc: 1.00
Epoch: 22, loss: 0.22362, train_acc: 1.00
Epoch: 23, loss: 0.20707, train_acc: 1.00
Epoch: 24, loss: 0.19545, train_acc: 1.00
E

#**Seq2Seq Model(N to M)**


In [8]:
'''
将一个序列转换为另一个序列。用RNN或者常用LSTM 或者GRU去避免梯度消失的问题。每个项目的上下文是上一步的输出。主要组件是一个编码器和一个解码器网络。
编码器将每个项目转换为包含该项目及其上下文的隐藏向量。
解码器使用先前的输出作为上下文来逆转该过程。将向量转换为输出项
'''

'\n将一个序列转换为另一个序列。用RNN或者常用LSTM 或者GRU去避免梯度消失的问题。每个项目的上下文是上一步的输出。主要组件是一个编码器和一个解码器网络。\n编码器将每个项目转换为包含该项目及其上下文的隐藏向量。\n解码器使用先前的输出作为上下文来逆转该过程。将向量转换为输出项\n'

**要开始一个新的项目了，该模型将纸牌符号（Ace，Jack，Queen，King） 转换为其相关编号**

In [9]:
'''
其实就两个模型一个是RNN，一个是Seq2Seq
1：处理数据。
2分批次
3模型
4评估

这个任务是"ace" 转变到"01" 将"jack"转变到"11"  【要求】老师
'''

'\n其实就两个模型一个是RNN，一个是Seq2Seq\n1：处理数据。\n2分批次\n3模型\n4评估\n\n这个任务是"ace" 转变到"01" 将"jack"转变到"11"  【要求】老师\n'

**Preprocess data**

In [10]:
'''
1调包
2 序列数据 sequence data
3 chars 字符  生成唯一的符号列表     tokens符号 
  每一个都拆开 + 去掉重复项
4 需要特殊的tokens 
    BEPU
    要额外添加
    B 序列开始
    E 序列结束
    P padding 填充（fill） 序列。 对于不同的输入大小
    U 序列中不知道的元素。 对于不同的输入大小

5 然后将字母和序号一一匹配上 ： n：i
6 记录整个长度
7 限定最大输入和输出的数量
'''

'\n1调包\n2 序列数据 sequence data\n3 chars 字符  生成唯一的符号列表     tokens符号 \n  每一个都拆开 + 去掉重复项\n4 需要特殊的tokens \n    BEPU\n    要额外添加\n    B 序列开始\n    E 序列结束\n    P padding 填充（fill） 序列。 对于不同的输入大小\n    U 序列中不知道的元素。 对于不同的输入大小\n\n5 然后将字母和序号一一匹配上 ： n：i\n6 记录整个长度\n7 限定最大输入和输出的数量\n'

In [2]:
import torch
import numpy as np
import pprint

seq_data = [['ace', '01'],['jack','11'],
            ['queen','12'],['king','13']]

chars =[] 
for seq in seq_data:
  chars += list(seq[0])
  chars += list(seq[1])

char_arr = list(set(chars)) #筛除重复的然后生成了列表 14

char_arr.append('B')
char_arr.append('E')
char_arr.append('P')
char_arr.append('U')    #18


num_dic = {n:i for i,n in enumerate(char_arr)} #对应字母：index 因为有0所以17 enumerate 就是让字母和数字一一对应

dic_len = len(num_dic)    #18

max_input_words_amount = 5
max_output_words_amount = 3

**产生batch--generate batch**

In [12]:
'''
1.padding (fill 补充) 添满如果单词比最长的单词要短
2.产生一批数据，给训练和测试
    创建空白列表；编码输入；解码输入； 目标批次
    然后开始输入，将字典的中的word输入到input中
 然后找到固定的每个单词对应的index列表 P就是16

[5, 1, 0, 16, 16]
[11, 5, 1, 4, 16]
[8, 2, 0, 0, 3]
[4, 7, 3, 6, 16]

    左边的输入准备完了，然后要进入右边了，要先来个B  B + index
[14, 13, 9]
[14, 9, 9]
[14, 9, 10]
[14, 9, 12]

    B将index也转换成列表了
    但之后就取消了B，加了E 来让index1又转换成列表
[13, 9, 15]
[9, 9, 15]
[9, 10, 15]
[9, 12, 15]   
  
  所以右边是解码器；
    解码器单元的输出（实际结果），在序列数据的末尾添加E
    由于目标长度是固定的，但实际上我们  不需要在最后加E
    但不确定的目标长度，E就很重要了。
    因为在预测的过程中当涉及到E我们就要停

   然后开始将准备好的编码和解码输入变成 one-hot (逃不掉的one-hot)

3 这里输入的编码和解码（b） 都变成了one-hot存起来了
只有target （e） 存的是index，并没有存one-hot
'''

#这里是把词表输入。然后呢处理他们，得到 解码编码-onehot ；和目标（可能就是真实）index

'\n1.padding (fill 补充) 添满如果单词比最长的单词要短\n2.产生一批数据，给训练和测试\n    创建空白列表；编码输入；解码输入； 目标批次\n    然后开始输入，将字典的中的word输入到input中\n 然后找到固定的每个单词对应的index列表 P就是16\n\n[5, 1, 0, 16, 16]\n[11, 5, 1, 4, 16]\n[8, 2, 0, 0, 3]\n[4, 7, 3, 6, 16]\n\n    左边的输入准备完了，然后要进入右边了，要先来个B  B + index\n[14, 13, 9]\n[14, 9, 9]\n[14, 9, 10]\n[14, 9, 12]\n\n    B将index也转换成列表了\n    但之后就取消了B，加了E 来让index1又转换成列表\n[13, 9, 15]\n[9, 9, 15]\n[9, 10, 15]\n[9, 12, 15]   \n  \n  所以右边是解码器；\n    解码器单元的输出（实际结果），在序列数据的末尾添加E\n    由于目标长度是固定的，但实际上我们  不需要在最后加E\n    但不确定的目标长度，E就很重要了。\n    因为在预测的过程中当涉及到E我们就要停\n\n   然后开始将准备好的编码和解码输入变成 one-hot (逃不掉的one-hot)\n\n3 这里输入的编码和解码（b） 都变成了one-hot存起来了\n只有target （e） 存的是index，并没有存one-hot\n'

In [3]:
def add_paddings(word, max_num=5):    #padding (fill 补充) 添满如果单词比最长的单词要短
  diff = max_num -len(word)
  return word + 'P'*diff    #用P补全  看起来要不补一个要不补两个P

def make_batch(seq_data):   #产生一批数据，给训练和测试
  encoder_input_batch = []
  decoder_input_batch = []
  target_batch = []

  for seq in seq_data:
    input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
    en_input_data = [num_dic[n] for n in input_word]          #[5, 1, 0, 16, 16]
    
    de_input_data = [num_dic[n] for n in ('B' + seq[1])]  #B：14， 13：0； 9：1 它是把idnex也变成了列表

    target = [num_dic[n] for n in (seq[1] + 'E')]       #   13:0      E:15
                                                        #[en_input_data]落了一个这个，后面就都不对了
    encoder_input_batch.append(np.eye(dic_len)[en_input_data]) #就是五位。让他们在对应位置表1；  5；8；4；7；17 （还是需要减一，是从0开始；所以单独输出的还是King「 
                                                #让他们input的word变成one-hot
    decoder_input_batch.append(np.eye(dic_len)[de_input_data])    #np.eye(总单词长度)【实际你想要的变得单词index】
                                                #还是包含B的index
    target_batch.append(target)                 #包含E的,他是直接将输出的index存起来了，没做one-hot


  return encoder_input_batch, decoder_input_batch, target_batch

#Model

In [14]:
#数据处理好了开始做模型了。
'''
1，导包
2，开始键模型nn模型来了
    有一层编码  RNN编码器
    有一层dropout tf.nn.dropout是TensorFlow里面为了防止或减轻过拟合而使用的函数，它一般用在全连接层。
      这里应该是pytorch的 dropout

        tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None,name=None)
        上面方法中常用的是前两个参数：
        第一个参数x：指输入
        第二个参数keep_prob: 设置神经元被选中的概率,在初始化时keep_prob是一个占位符, keep_prob = tf.placeholder(tf.float32) 。tensorflow在run时设置keep_prob具体的值，例如keep_prob: 0.5
        第五个参数name：指定该操作的名字。
      drop out 可以作为一个层，在RNN输出中，请见下面的forward
      以前我们将其作为nn.RNN 的参数（dropout= 0.2）

      RNN 解码器 就比编码器多了一层线性
'''

'''
开始向前传播了。
隐藏层，包含隐藏状态对于 t = seq_len
应用 dropout 层在 输出的RNN上

设置隐藏作为 rnn解码器的初始状态。【很重要】

模型的原理有点混乱需要重新看KDB或者Caren的课程。



# prediction_output_before_softmax = self.linear(decoder_output)  也就是我们只需要这部了，因为nn.CrossEntropyLoss 结合了 logsoftmax 和Nllloss
# output_after_softmax = torch.log_softmax(prediction_output_before_softmax,dim=-1)
# Since nn.CrossEntropyLoss combines LogSoftmax and NLLLoss for us, we only need the prediction_output_before_softmax

        '''

'\n开始向前传播了。\n隐藏层，包含隐藏状态对于 t = seq_len\n应用 dropout 层在 输出的RNN上\n\n设置隐藏作为 rnn解码器的初始状态。【很重要】\n\n模型的原理有点混乱需要重新看KDB或者Caren的课程。\n\n\n\n# prediction_output_before_softmax = self.linear(decoder_output)  也就是我们只需要这部了，因为nn.CrossEntropyLoss 结合了 logsoftmax 和Nllloss\n# output_after_softmax = torch.log_softmax(prediction_output_before_softmax,dim=-1)\n# Since nn.CrossEntropyLoss combines LogSoftmax and NLLLoss for us, we only need the prediction_output_before_softmax\n\n        '

In [15]:
#setting hyperparameters
'''
设置超参开始训练
然后训练
'''

'\n设置超参开始训练\n然后训练\n'

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score


class Seq2Seq_Model(nn.Module):
  def __init__(self):
    super(Seq2Seq_Model, self).__init__()

    self.rnn_encoder = nn.RNN(n_input, n_hidden, batch_first = True)    #RNN编码器 encoder  #self.rnn_encoder 不是self.rnn.encoder
    self.dropout_encoder = nn.Dropout(0.1)    #我认为应该是p = 0.1   # dropout 可以作为一个层
    
                                              #RNN decoder
    self.rnn_decoder = nn.RNN(n_input, n_hidden, batch_first = True)
    self.dropout_decoder = nn.Dropout(0.1)
    self.linear = nn.Linear(n_hidden,n_class) #多了层线性

  def forward(self, x_encoder, x_decoder):
    _,hidden = self.rnn_encoder(x_encoder)    # 下划线+'，'的作用是：使预测返回的output.data行中最大数值所在位置代表的类别。_,predicted=torch.max(output.data,dim=1)
    hidden = self.dropout_encoder(hidden)   #输出上用dropout层

                                            #***** 设置隐藏作为 rnn解码器的初始状态。
    decoder_output,_ = self.rnn_decoder(x_decoder,hidden)
    decoder_output = self.dropout_decoder(decoder_output)      #输出上用dropout层 和上个hidden的功能描述一样

    output = self.linear(decoder_output)

    return output
'''
然后要输入超参开始训练了
'''

learning_rate = 0.01
n_hidden = 128
total_epoch = 200

n_class = dic_len
n_input = dic_len

model = Seq2Seq_Model().to(device)    ## to(device) 将输入转换为张量，并将其设置成为GPU 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

encoder_input_batch, decoder_input_batch, target_batch = make_batch(seq_data) 
encoder_input_torch = torch.from_numpy(np.array(encoder_input_batch)).float().to(device)  #4位张量目前对角都是1 [4,18,18]
decoder_input_torch = torch.from_numpy(np.array(decoder_input_batch)).float().to(device)  #list---array【没跑之前】跑之后，【4，3，18】
target_batch_torch = torch.from_numpy(np.array(target_batch)).view(-1).to(device) # (-1)自行计算行数或列数
                                                                        #把所有的index都合在一个tensor里了。[12]

for epoch in range(total_epoch):      #循环数据集多次
  model.train()
  optimizer.zero_grad()

  #forward + backward + optimize
  outputs = model(encoder_input_torch,decoder_input_torch)
  loss = criterion(outputs.view(-1,outputs.size(-1)), target_batch_torch)   #tensor(2.8513, 
  loss.backward()
  optimizer.step()

  if epoch%10==9:   #每10个
    print('Epoch: %d, loss: %.5f' %(epoch +1, loss.item()))

  
print('Finished Training')
#loss 每次都不同

Epoch: 10, loss: 0.65843
Epoch: 20, loss: 0.51416
Epoch: 30, loss: 0.34315
Epoch: 40, loss: 0.32002
Epoch: 50, loss: 0.29404
Epoch: 60, loss: 0.31863
Epoch: 70, loss: 0.16016
Epoch: 80, loss: 0.10391
Epoch: 90, loss: 0.13731
Epoch: 100, loss: 0.07034
Epoch: 110, loss: 0.01164
Epoch: 120, loss: 0.00538
Epoch: 130, loss: 0.00415
Epoch: 140, loss: 0.00190
Epoch: 150, loss: 0.00120
Epoch: 160, loss: 0.00110
Epoch: 170, loss: 0.00095
Epoch: 180, loss: 0.00056
Epoch: 190, loss: 0.00065
Epoch: 200, loss: 0.00055
Finished Training


#Evaluation


In [17]:
#开始评估了
'''
定义预测的单词
  增添填充满
  设置每个预测的角色为不知道U
  【'king'（padded），'UU'】
预测第一个token
预测第二个token  标记
'''

In [5]:
def predict(word):
  word = add_paddings(word, max_input_words_amount)   #所以这里的word传的是谁呢？

  seq_data = [word, 'U' * 2]

  encoder_input_batch, decoder_input_batch, _ = make_batch([seq_data])
  encoder_input_torch = torch.from_numpy(np.array(encoder_input_batch)).float().to(device)
  decoder_input_torch = torch.from_numpy(np.array(decoder_input_batch)).float().to(device)

  model.eval()
  outputs = model(encoder_input_torch, decoder_input_torch)
  predicted = torch.argmax(outputs, -1)
  first_token = char_arr[predicted.cpu().numpy()[0][0]]
  
  seq_data[1] = first_token + 'U' #预测第二个标记

  encoder_input_batch, decoder_input_batch, _ = make_batch([seq_data])
  encoder_input_torch = torch.from_numpy(np.array(encoder_input_batch)).float().to(device)
  decoder_input_torch = torch.from_numpy(np.array(decoder_input_batch)).float().to(device)

  model.eval()
  outputs = model(encoder_input_torch,decoder_input_torch)
  predicted = torch.argmax(outputs, -1)
  second_token = char_arr[predicted.cpu().numpy()[0][1]]

  return first_token + second_token

print('=== Prediction result ===')
print('ace ->', predict('ace'))
print('jack ->', predict('jack'))
print('queen ->', predict('queen'))
print('king ->', predict('king'))

=== Prediction result ===
ace -> 01
jack -> 11
queen -> 12
king -> 13


**Text**

In [2]:
# Task is "ace" -> "01", "jack" -> "11",  a sequence to sequence problem with teacher forcing

import torch
import numpy as np


# Sequence data
seq_data = [['ace', '01'], ['jack', '11'],
            ['queen', '12'], ['king', '13']]

# Generate unique tokens list
chars = []
for seq in seq_data:    
    chars += list(seq[0])
    chars += list(seq[1])

# To simplify the question, we put all characters (including input and output) into one set
char_arr = list(set(chars))

# special tokens are required
# B: Beginning of Sequence
# E: Ending of Sequence
# P: Padding of Sequence - for different size input
# U: Unknown element of Sequence - for different size input
char_arr.append('B')
char_arr.append('E')
char_arr.append('P')
char_arr.append('U')

num_dic = {n: i for i, n in enumerate(char_arr)}

dic_len = len(num_dic)

max_input_words_amount = 5
max_output_words_amount = 3

In [3]:
# add paddings if the word is shorter than the maximum number of words
def add_paddings(word, max_num=5):
    diff = max_num - len(word)
    return word+'P'*diff
    

# generate a batch data for training/testing
def make_batch(seq_data):
    encoder_input_batch = []
    decoder_input_batch = []
    target_batch = []

    for seq in seq_data:
        # Input for encoder cell, convert to vector
        input_word = add_paddings(seq[0])
        en_input_data = [num_dic[n] for n in input_word]
        
        # Input for decoder cell, Add 'B' at the beginning of the sequence data
        de_input_data  = [num_dic[n] for n in ('B'+ seq[1])]
        
        # Output of decoder cell (Actual result), Add 'E' at the end of the sequence data
        # Since the length of target is fixed, actually we don't need to add "E" in the end
        # However, for uncertain target length, 'E' is important since we need to stop when it comes to "E" during the prediction
        target = [num_dic[n] for n in (seq[1] + 'E')]

        # Convert each character vector to one-hot encoding data
        encoder_input_batch.append(np.eye(dic_len)[en_input_data])
        decoder_input_batch.append(np.eye(dic_len)[de_input_data])
        
        target_batch.append(target)
        

    return encoder_input_batch, decoder_input_batch, target_batch

    #【这的问题】

In [1]:
#recurrent （复发的 蕊·科·run3·忒） sequence （计数，序数） （C 困·嘶） neural（牛肉）
#RNN ； seq2seq
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# you can GPU(cuda)/ just CPU

In [34]:
#6.1
seq_data

[['ace', '01'], ['jack', '11'], ['queen', '12'], ['king', '13']]

In [18]:
#5.6
loss = criterion(outputs.view(-1,outputs.size(-1)), target_batch_torch)
loss


tensor(0.5444, device='cuda:0', grad_fn=<NllLossBackward>)

In [19]:
#5.5
target_batch_torch

tensor([12,  5, 15,  5,  5, 15,  5,  6, 15,  5,  7, 15], device='cuda:0')

In [20]:
#5.4
outputs.size(-1)

18

In [21]:
#5.3
outputs.view(-1,outputs.size(-1)) #12

tensor([[ -5.3477,  -5.8705,  -5.9985,  -5.6836,  -5.6007,   7.7865,  -0.2808,
          -0.0754,  -5.3091,  -6.4393,  -5.2367,  -5.7612,   6.0992,  -6.0102,
          -6.4937,  -3.2540,  -6.0402,  -5.6420],
        [ -8.1359,  -8.3976,  -8.7912,  -8.4217,  -7.7891,   8.7916,   5.1923,
           5.2890,  -7.7396,  -8.4006,  -8.0030,  -8.6032,   1.3184,  -8.2034,
          -9.1096,  -4.1776,  -7.7197,  -7.9167],
        [ -2.2059,  -2.6412,  -2.8927,  -3.0924,  -2.2640,  -2.4952,   2.9284,
           4.2778,  -2.7378,  -3.1537,  -2.4296,  -2.9000,  -1.4073,  -3.9331,
          -3.6077,  14.3211,  -2.3606,  -3.0859],
        [ -4.5275,  -5.6051,  -4.8683,  -5.2986,  -4.9559,   8.1549,  -0.1552,
          -0.3142,  -4.6840,  -6.0729,  -4.4751,  -5.3116,   4.7771,  -5.2472,
          -5.6654,  -4.0313,  -5.8738,  -4.7678],
        [ -7.5813,  -8.3008,  -8.6082,  -7.9169,  -7.6968,   6.7230,   7.1202,
           7.9293,  -7.8273,  -7.4920,  -7.9478,  -8.4173,   0.9290,  -8.1355,
          

In [22]:
#5.2
outputs.view(-1)

tensor([ -5.3477,  -5.8705,  -5.9985,  -5.6836,  -5.6007,   7.7865,  -0.2808,
         -0.0754,  -5.3091,  -6.4393,  -5.2367,  -5.7612,   6.0992,  -6.0102,
         -6.4937,  -3.2540,  -6.0402,  -5.6420,  -8.1359,  -8.3976,  -8.7912,
         -8.4217,  -7.7891,   8.7916,   5.1923,   5.2890,  -7.7396,  -8.4006,
         -8.0030,  -8.6032,   1.3184,  -8.2034,  -9.1096,  -4.1776,  -7.7197,
         -7.9167,  -2.2059,  -2.6412,  -2.8927,  -3.0924,  -2.2640,  -2.4952,
          2.9284,   4.2778,  -2.7378,  -3.1537,  -2.4296,  -2.9000,  -1.4073,
         -3.9331,  -3.6077,  14.3211,  -2.3606,  -3.0859,  -4.5275,  -5.6051,
         -4.8683,  -5.2986,  -4.9559,   8.1549,  -0.1552,  -0.3142,  -4.6840,
         -6.0729,  -4.4751,  -5.3116,   4.7771,  -5.2472,  -5.6654,  -4.0313,
         -5.8738,  -4.7678,  -7.5813,  -8.3008,  -8.6082,  -7.9169,  -7.6968,
          6.7230,   7.1202,   7.9293,  -7.8273,  -7.4920,  -7.9478,  -8.4173,
          0.9290,  -8.1355,  -8.8589,  -2.6555,  -7.7854,  -7.93

In [23]:
#5.1
outputs = model(encoder_input_torch,decoder_input_torch)
outputs

tensor([[[-3.9173e+00, -5.5697e+00, -5.2363e+00, -5.0532e+00, -5.0746e+00,
           7.2907e+00, -1.6714e+00, -1.8430e-01, -4.9302e+00, -5.3972e+00,
          -4.6389e+00, -5.1971e+00,  5.6213e+00, -5.6339e+00, -5.5697e+00,
          -3.3276e+00, -5.5813e+00, -4.8254e+00],
         [-7.6901e+00, -8.1738e+00, -8.5171e+00, -8.0193e+00, -7.9836e+00,
           9.1194e+00,  3.4588e+00,  4.1480e+00, -7.4914e+00, -8.3521e+00,
          -7.7683e+00, -8.1236e+00,  2.2012e+00, -8.0564e+00, -8.3892e+00,
          -3.0812e+00, -7.5897e+00, -7.5448e+00],
         [-2.1666e+00, -2.9426e+00, -3.0201e+00, -3.4295e+00, -2.2210e+00,
          -2.3405e+00,  1.7433e+00,  3.6801e+00, -2.7634e+00, -3.3306e+00,
          -2.7341e+00, -2.8670e+00, -8.2571e-01, -3.5560e+00, -3.1427e+00,
           1.4274e+01, -2.2170e+00, -2.7895e+00]],

        [[-4.7471e+00, -5.4746e+00, -5.2784e+00, -5.4372e+00, -5.2484e+00,
           7.1052e+00,  6.1981e-02, -4.1968e-01, -4.9605e+00, -6.2743e+00,
          -4.7935e+00, 

In [24]:
target_batch_torch.shape

torch.Size([12])

In [25]:
target_batch_torch

tensor([12,  5, 15,  5,  5, 15,  5,  6, 15,  5,  7, 15], device='cuda:0')

In [26]:
target_batch

[[12, 5, 15], [5, 5, 15], [5, 6, 15], [5, 7, 15]]

In [27]:
decoder_input_batch.shape

AttributeError: ignored

In [ ]:
encoder_input_batch.shape

In [ ]:
decoder_input_batch

In [ ]:
encoder_input_batch

In [ ]:
target_batch.append(target)
target_batch

In [ ]:
#4.18
target

In [ ]:
dic_len

In [ ]:
np.eye(dic_len)[de_input_data]  #14, 9 ,12

In [ ]:
en_input_data

In [ ]:
#4.14
np.eye(dic_len)[en_input_data] #kingP 

In [ ]:
#4.13
np.eye(dic_len)

In [ ]:
#4.12
encoder_input_batch = []
decoder_input_batch = []
target_batch = []

for seq in seq_data:
  input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
  en_input_data = [num_dic[n] for n in input_word]          #[5, 1, 0, 16, 16]
  
  de_input_data = [num_dic[n] for n in ('B' + seq[1])]  #B：14， 13：0； 9：1 它是把idnex也变成了列表

  target = [num_dic[n] for n in (seq[1] + 'E')]
  print(target)

In [ ]:
#4.11
seq

In [ ]:
#4.10
encoder_input_batch = []
decoder_input_batch = []
target_batch = []

for seq in seq_data:
  input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
  en_input_data = [num_dic[n] for n in input_word]
  de_input_data = [num_dic[n] for n in ('B' + seq[1])]
  print(de_input_data)    

In [ ]:
#4.9
encoder_input_batch = []
decoder_input_batch = []
target_batch = []

for seq in seq_data:
  input_word = add_paddings(seq[0])     # 0 指前面的word 1 指后面的index  kingP
  en_input_data = [num_dic[n] for n in input_word]
  print(en_input_data)    


In [ ]:
#4.8 
n

In [ ]:
#4.7
len(num_dic)

In [ ]:
#4.6
num_dic[n]

In [ ]:
#4.5
num_dic

In [ ]:
input_word = add_paddings(seq[0])
input_word

In [ ]:
seq[1]

In [ ]:
#4.2
seq[0]

In [ ]:
#4.1
seq_data

In [ ]:
#3.1
chars += list(seq[0])
'''
['a', 'c', 'e']
['a', 'c', 'e', 'j', 'a', 'c', 'k']
['a', 'c', 'e', 'j', 'a', 'c', 'k', 'q', 'u', 'e', 'e', 'n']
['a', 'c', 'e', 'j', 'a', 'c', 'k', 'q', 'u', 'e', 'e', 'n', 'k', 'i', 'n', 'g']
'''
chars += list(seq[1])
'''
['a', 'c', 'e', '0', '1']
['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1']
['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1', 'q', 'u', 'e', 'e', 'n', '1', '2']
['a', 'c', 'e', '0', '1', 'j', 'a', 'c', 'k', '1', '1', 'q', 'u', 'e', 'e', 'n', '1', '2', 'k', 'i', 'n', 'g', '1', '3']

'''

In [ ]:
#2.17
seq_data

In [ ]:
#2.16
char_arr

In [ ]:
#2.15
char_arr[ind]

In [ ]:
#2.14
'''
['wor']
['wor', 'woo']
['wor', 'woo', 'dee']
['wor', 'woo', 'dee', 'div']
['wor', 'woo', 'dee', 'div', 'col']
['wor', 'woo', 'dee', 'div', 'col', 'coo']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis']
['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
'''

#seq_data[i][:-1]


In [ ]:
#2.13
ind

In [ ]:
#2.12
predicted.cpu()[1]

In [ ]:
#2.11
predicted.cpu().numpy()

In [ ]:
#2.10
predicted.cpu()

In [ ]:
#2.9
predicted

In [ ]:
#2.8
target_batch_torch.cpu().numpy()

In [ ]:
#2.7
target_batch_torch

In [ ]:
#2.6
print(target_batch_torch.cpu())#tensor([ 3,  3, 15,  4,  3, 11,  3,  4, 18,  3])


In [ ]:
#2.5
  #print(_)    #tensor([0.2457, 0.1858, 0.2913, 0.1972, 0.2256, 0.2471, 0.2616, 0.1699, 0.2519,
              #0.2905], device='cuda:0', grad_fn=<MaxBackward0>)  [10]torch.Size([10]) 50+(不会是64吧）)
  #print(len(_))
  #print(predicted) #tensor([ 4,  4,  4,  4,  4,  4,  4,  4, 10,  4], device='cuda:0')
  #print(predicted.shape) #torch.Size([10])

In [ ]:
#2.4
target_batch_torch

In [ ]:
#2.3
target_batch

In [ ]:
#2.2
model.train()

In [ ]:
#2.1
input_batch_torch

In [ ]:
#1.9
'''
target_batch = []
for seq in seq_data:
  input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来
  target = num_dic[seq[-1]]

  input_batch.append(np.eye(dic_len)[input_data])

  target_batch.append([target])
  print(target_batch)
'''

In [ ]:
#1.8
'''
import pprint
input_batch = []
for seq in seq_data:
  input_data = [num_dic[n] for n in seq[:-1]] ##不取最后一个，把前面的字母对应的index取出来

  input_batch.append(np.eye(dic_len)[input_data]) #2-3【0】【0】 还是个list；还没有形状0 0
  

'''

In [ ]:
#np.eye(dic_len)[input_data] #制作one-hot np.eye 批量制作。然后第一个是0.所以index+1的位置是1.代表了这个元素

In [ ]:
#input_data #kid

In [ ]:
np.eye(dic_len)#弄了个对角线1其他都是0


In [ ]:
#1.7
'''
for seq in seq_data:
  target = num_dic[seq[-1]]
  print(target)     #3 :d  
  '''

In [ ]:
#1.6
'''
for seq in seq_data:
    input_data = [num_dic[n] for n in seq[:-1]] #不取最后一个
    print(input_data)
    #w:22 o:14 r:17
    '''

In [ ]:
#1.5
seq_data
len(seq_data)

In [ ]:
#1.4
dic_len

In [ ]:
#1.3
{n:i for i, n in enumerate(char_arr)} #   n: i == 'a': 0,单词： index 目前看起来只有字典可以这样

In [ ]:
#1.2
for i,n in enumerate(char_arr):
  #print(i,)    #index
  #print(n)    #word
  print(i,n)  #这样是一一对应（横向）

In [ ]:
#1.1
list(enumerate(char_arr))#这没什么变化啊
